## Building, Deploying, Connecting, Optimizing, and Extending AI Agents

This notebook shows the full lifecycle of creating an advanced AI agent using Amazon Bedrock AgentCore and Strands.

### 1. Local Agent Development
Build a simple Strands Agent, add small tools, and run it locally.  
This is the fast, iterative phase where the agent’s core behavior is validated.

### 2. Preparing for Runtime Deployment
Wrap the agent with an AgentCore Runtime entrypoint, generate a container, deploy it, and invoke it.  
The agent becomes a managed, scalable service.

### 3. Introducing AgentCore Gateway
Gateway provides secure access to external tools, identity isolation, protocol translation, and built-in tool search.  
It becomes essential when the agent must interact with many services.

### 4. Creating Gateways and Adding Tools
Create a Gateway, configure identity, register tools and schemas, expose APIs, and scale up to large tool catalogs.  
The Gateway acts as the unified tool layer for the agent.

### 5. Semantic Tool Search
Instead of giving the agent hundreds of tools, use natural language search to retrieve only the relevant ones.  
This reduces complexity and boosts accuracy.

### 6. Using Gateway Tools Inside a Strands Agent
Integrate the Gateway with the agent.  
Use either the full toolset or only the search-filtered subset.  
Dynamic tool selection leads to cleaner reasoning and fewer errors.

### 7. Performance and Token Savings
Compare two styles:
- an agent with all tools  
- an agent using only search-selected tools  

Search-based selection consistently reduces latency, token usage, and reasoning overhead.

### 8. Adding Memory
Add AgentCore Memory with a summary strategy and custom hooks.  
The agent retrieves relevant past context and stores new interactions automatically.  
This enables personalization and continuity.

### 9. Conclusion
The final agent:
- runs locally and in Runtime  
- uses Gateway for tool access  
- picks tools intelligently through search  
- performs faster with fewer tokens  
- remembers past interactions through Memory  

This represents a complete, practical blueprint for production-grade AI agents.

## Prerequisites

To execute this tutorial you will need:

* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents


In [49]:
from dotenv import load_dotenv
import os

aws_region = "us-west-2"
load_dotenv(".env")

os.environ["AWS_REGION"] = aws_region
os.environ["AWS_DEFAULT_REGION"] = aws_region

In [47]:
session = boto3.Session()
region = session.region_name

print(region)

None


In [7]:
!pip install --force-reinstall -U -r requirements.txt --quiet

/Users/gianluigimucciolo/.zshenv:.:1: no such file or directory: /Users/gianluigimucciolo/.cargo/env


## Creating your agent and experimenting locally

Before we deploy our agent to AgentCore Runtime, let's develop and run it locally for experimentation purposes.

For production agentic applications we will need to decouple the agent creation process from the agent invocation one. With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. Let's first look at how the agent is developed during the experimentation phase.

The local architecture will look as follows:

![Local architecture](images/architecture_local.png)


In [8]:
%%writefile strands_claude_agent.py
from strands import Agent, tool
from strands_tools import calculator
from strands.models import BedrockModel
import json
import argparse

@tool
def weather():
    """Get weather"""
    return "sunny"

model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(
    model_id=model_id,
)

agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather.",
)

def strands_agent_bedrock(payload):
    """Invoke the agent with a payload."""
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message["content"][0]["text"]

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    result = strands_agent_bedrock(json.loads(args.payload))
    print(result)


Overwriting strands_claude_agent.py


#### Invoking local agent from the notebook

We can now run our agent locally by calling the Python file from the notebook.


In [9]:
!python strands_claude_agent.py '{"prompt": "What is the weather now?"}'


/Users/gianluigimucciolo/.zshenv:.:1: no such file or directory: /Users/gianluigimucciolo/.cargo/env
I can check the current weather for you. Let me do that right away.
Tool #1: weather
The current weather is sunny! It's a beautiful day outside.The current weather is sunny! It's a beautiful day outside.


## Preparing your agent for deployment on AgentCore Runtime

Now let's deploy our agent to AgentCore Runtime. Notice that we are **reusing the same agent code** we just created in `strands_claude_agent.py`.

To expose this existing agent to AgentCore Runtime we only need to:

* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate a thin wrapper function that calls our existing `strands_agent_bedrock` with the `@app.entrypoint` decorator
* Let AgentCore Runtime control the running of the agent with `app.run()`

This keeps a clear separation between:
* **Agent definition and local experimentation** (`strands_claude_agent.py`)
* **Runtime wiring for deployment** (`strands_claude_runtime.py`)


In [10]:
%%writefile strands_claude_runtime.py
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands_claude_agent import strands_agent_bedrock

app = BedrockAgentCoreApp()

@app.entrypoint
def invoke_runtime(payload: dict):
    """AgentCore Runtime entrypoint that reuses the existing agent."""
    return strands_agent_bedrock(payload)

if __name__ == "__main__":
    app.run()


Overwriting strands_claude_runtime.py


## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on port `8080`
* Implements the required `/invocations` endpoint for processing the agent's requests
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to AWS standards


## Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent.

> **Note:** Operations best practice is to package code as a container and push to ECR using CI/CD pipelines and IaC.

In this tutorial we will use the Amazon Bedrock AgentCore Python starter toolkit to easily package your artifacts and deploy them to AgentCore Runtime.


### Configure AgentCore Runtime deployment

First we will use the starter toolkit to configure the AgentCore Runtime deployment with:

* An entrypoint file (`strands_claude_runtime.py`)
* An execution role (auto-created)
* A requirements file
* Auto-created Amazon ECR repository on launch

During the configure step, your Dockerfile will be generated based on your application code.

![Configure runtime](images/configure.png)


In [14]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "strands_claude_getting_started"

configure_result = agentcore_runtime.configure(
    entrypoint="strands_claude_runtime.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements_agent.txt",
    region=region,
    agent_name=agent_name,
)

configure_result


Entrypoint parsed: file=/Users/gianluigimucciolo/Documents/Progetti/genai-workshop/sd-workshop/serverless-day-genai-workshop/strands_claude_runtime.py, bedrock_agentcore_name=strands_claude_runtime
Configuring BedrockAgentCore agent: strands_claude_getting_started
Generated Dockerfile: /Users/gianluigimucciolo/Documents/Progetti/genai-workshop/sd-workshop/serverless-day-genai-workshop/Dockerfile
Generated .dockerignore: /Users/gianluigimucciolo/Documents/Progetti/genai-workshop/sd-workshop/serverless-day-genai-workshop/.dockerignore
Keeping 'strands_claude_getting_started' as default agent
Bedrock AgentCore configured: /Users/gianluigimucciolo/Documents/Progetti/genai-workshop/sd-workshop/serverless-day-genai-workshop/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/Users/gianluigimucciolo/Documents/Progetti/genai-workshop/sd-workshop/serverless-day-genai-workshop/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/Users/gianluigimucciolo/Documents/Progetti/genai-workshop/sd-workshop/serverless-day-genai-workshop/Dockerfile'), dockerignore_path=PosixPath('/Users/gianluigimucciolo/Documents/Progetti/genai-workshop/sd-workshop/serverless-day-genai-workshop/.dockerignore'), runtime='Docker', region='us-west-2', account_id='684359859465', execution_role=None, ecr_repository=None, auto_create_ecr=True)

### Launching agent to AgentCore Runtime

Now that we've got a Dockerfile, let's launch the agent to AgentCore Runtime. This will create (if needed) the Amazon ECR repository and the AgentCore Runtime.

![Launch runtime](images/launch.png)


In [15]:
launch_result = agentcore_runtime.launch()
launch_result


🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'strands_claude_getting_started' to account 684359859465 (us-west-2)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: strands_claude_getting_started
✅ ECR repository available: 684359859465.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_claude_getting_started
Getting or creating execution role for agent: strands_claude_getting_started
Using AWS region: us-west-2, account ID: 684359859465
Role name: AmazonBedrockAgentCoreSDKRuntime-us-west-2-8556fc4504


✅ Reusing existing ECR repository: 684359859465.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_claude_getting_started


✅ Reusing existing execution role: arn:aws:iam::684359859465:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-8556fc4504
✅ Execution role available: arn:aws:iam::684359859465:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-8556fc4504
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: strands_claude_getting_started
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-8556fc4504
Reusing existing CodeBuild execution role: arn:aws:iam::684359859465:role/AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-8556fc4504
Using .dockerignore with 44 patterns
Uploaded source to S3: strands_claude_getting_started/source.zip
Updated CodeBuild project: bedrock-agentcore-strands_claude_getting_started-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 1s)
✅ QUEUED completed in 1.5s
🔄 PROVISIONING started (total: 2s)
✅ PROVISIONING completed in 7.7s
🔄 DOWNLOAD_SOURCE started (t

LaunchResult(mode='codebuild', tag='bedrock_agentcore-strands_claude_getting_started:latest', env_vars=None, port=None, runtime=None, ecr_uri='684359859465.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_claude_getting_started', agent_id='strands_claude_getting_started-xfzR3RETfu', agent_arn='arn:aws:bedrock-agentcore:us-west-2:684359859465:runtime/strands_claude_getting_started-xfzR3RETfu', codebuild_id='bedrock-agentcore-strands_claude_getting_started-builder:fa144d73-406e-486c-95be-720738e4037d', build_output=None)

### Checking AgentCore Runtime status

After deploying the AgentCore Runtime, we can poll for its deployment status.


In [16]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint["status"]
end_status = ["READY", "CREATE_FAILED", "DELETE_FAILED", "UPDATE_FAILED"]

while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint["status"]
    print(status)

status


Retrieved Bedrock AgentCore status for: strands_claude_getting_started


'READY'

### Invoking AgentCore Runtime from the starter toolkit

Once the runtime is `READY`, we can invoke it with a JSON payload.


In [20]:
invoke_response = agentcore_runtime.invoke({"prompt": "How is the weather now?"})
invoke_response


{'ResponseMetadata': {'RequestId': 'd1396966-2f88-4206-bc0f-cbc900a9a828',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 22 Nov 2025 17:34:59 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd1396966-2f88-4206-bc0f-cbc900a9a828',
   'baggage': 'Self=1-6921f440-10b56ce40f7f48a507805de9,session.id=3cd5f417-b751-4456-8804-dc279386d005',
   'x-amzn-bedrock-agentcore-runtime-session-id': '3cd5f417-b751-4456-8804-dc279386d005',
   'x-amzn-trace-id': 'Root=1-6921f440-27215b024697db6329c847c3;Parent=191b5630af0d2301;Sampled=1;Self=1-6921f440-10b56ce40f7f48a507805de9'},
  'RetryAttempts': 0},
 'runtimeSessionId': '3cd5f417-b751-4456-8804-dc279386d005',
 'traceId': 'Root=1-6921f440-27215b024697db6329c847c3;Parent=191b5630af0d2301;Sampled=1;Self=1-6921f440-10b56ce40f7f48a507805de9',
 'baggage': 'Self=1-6921f440-10b56ce40f7f48a507805de9,session.id=3cd5f417-b751-4456-8804-dc279386d005',
 'contentType': 

### Processing invocation results

We can now process our invocation results to include them in an application.


In [22]:
from IPython.display import Markdown, display

response_text = invoke_response["response"][0]
display(Markdown(response_text))


The weather right now is sunny! It's a beautiful day outside.

### Invoking AgentCore Runtime with boto3

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method.


In [27]:
status_response = agentcore_runtime.status()
print(f"Status: {status_response.endpoint['status']}")
print(f"Full response: {status_response}")

Retrieved Bedrock AgentCore status for: strands_claude_getting_started


Status: READY
Full response: config=StatusConfigInfo(name='strands_claude_getting_started', entrypoint='/Users/gianluigimucciolo/Documents/Progetti/genai-workshop/sd-workshop/serverless-day-genai-workshop/strands_claude_runtime.py', region='us-west-2', account='684359859465', execution_role='arn:aws:iam::684359859465:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-8556fc4504', ecr_repository='684359859465.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_claude_getting_started', agent_id='strands_claude_getting_started-xfzR3RETfu', agent_arn='arn:aws:bedrock-agentcore:us-west-2:684359859465:runtime/strands_claude_getting_started-xfzR3RETfu') agent={'ResponseMetadata': {'RequestId': '62fe02af-77bf-4e1f-abd9-0ffb17934afb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 22 Nov 2025 17:38:17 GMT', 'content-type': 'application/json', 'content-length': '1011', 'connection': 'keep-alive', 'x-amzn-requestid': '62fe02af-77bf-4e1f-abd9-0ffb17934afb', 'x-amzn-remapped-x-amzn-request

In [29]:
import boto3
import json

agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    "bedrock-agentcore",
    region_name=aws_region,
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is 2+2?"}),
)

if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))


The answer to 2+2 is 4.

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime and ECR repository created for this tutorial.


In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split("/")[1]


In [ ]:
import boto3

agentcore_control_client = boto3.client(
    "bedrock-agentcore-control",
    region_name=region,
)
ecr_client = boto3.client(
    "ecr",
    region_name=region,
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split("/")[1],
    force=True,
)


# Congratulations!

You have:

* Built and tested a Strands agent locally
* Reused the same agent code as an entrypoint for Amazon Bedrock AgentCore Runtime
* Deployed, invoked, and cleaned up the runtime using both the starter toolkit and boto3


# Amazon Bedrock AgentCore Gateway - Semantic search tutorial

### Tutorial Details


| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                                   |
| Agent type          | Single                                                                           |
| AgentCore services  | AgentCore Gateway, AgentCore Identity                                            |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Sonnet 3.7                                                        |
| Tutorial components | Creating and using Lambda-backed AgentCore Gateway from Strands Agent            |
| Tutorial vertical   | Cross-vertical                                                                   |
| Example complexity  | Easy                                                                             |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3  

### Tutorial Architecture
Amazon Bedrock AgentCore Gateway provides unified connectivity between agents and the tools and resources they need to interact with. Gateway plays multiple roles in this connectivity layer:

1. **Security Guard**: Gateway manages OAuth authorization to ensure only valid users / agents access tools / resources.
2. **Translator**: Gateway translates agent requests made using popular protocols like the Model Context Protocol (MCP) into API requests and Lambda invocations. This means developers don’t need to host servers, manage protocol integration, version support, version patching, etc.
3. **Composer**: Gateway enables developers to seamlessly combine multiple APIs, functions, and tools into a single MCP  endpoint that an agent can use.
4. **Keychain**: Gateway handles the injection of the right credentials to use with the right tool, ensuring that agents can seamlessly leverage tools that require different sets of credentials. 
5. **Researcher**: Gateway enables agents to search across all of their tools to find only the ones that are best for a given context or question. This allows agents to make use of 1000s of tools instead of just a handful. It also minimizes the set of tools that need to be provided in an agent’s LLM prompt, reducing latency and cost. 
6. **Infrastructure Manager**: Gateway is completely serverless, and comes with built-in observability and auditing, alleviating the need for developers to manage additional infrastructure to integrate their agents and tools. 

![How does it work](images/gw-arch-overview.png)

### Tutorial Key Features

* Creating Amazon Bedrock AgentCore Gateways with AWS Lambda-backed targets
* Using AgentCore Gateway semantic search 
* Using Strands Agents to show how AgentCore Gateway search improves latency

## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents

## AgentCore Gateway helps solve the challenge of MCP servers that have large numbers of tools
In a typical enterprise setting, agent builders encounter MCP servers that have hundreds or even thousands
of MCP tools. This volume of tools poses challenges for AI agents, including poor tool selection accuracy, 
increased cost, and higher latency driven by higher token usage from excessive tool metadata.
This can happen when connecting your agents to third party services (e.g., Zendesk, Salesforce,
Slack, JIRA, ...), or to existing enterprise REST services. 
AgentCore Gateway provides a built in semantic search across tools, 
which improves agent latency, cost, and accuracy, while still giving those agents the tools they need. 
Depending on your use case, LLM model, and agent framework, you can see up to 3x better latency by keeping
an agent focused on relevant tools versus providing the full set of hundreds of tools from a typical MCP Server.

![How does it work](images/gateway_tool_search.png)

## What you will learn in this notebook
In this notebook, we provide a tutorial for AgentCore Gateway search. By the end of this step-by-step tutorial, you
will understand:

- How to use AgentCore Gateway's built-in search tool to quickly find relevant tools 
- How to integrate tool search results into Strands Agents for improved latency and reduced cost

## Overview of the notebook structure
The notebook is structured with the following sections:

1. Understanding fundamentals of AgentCore Gateway Search
2. Preparing the notebook environment
3. Setting up a Gateway that has hundreds of tools
4. Searching for tools from a Gateway
5. Using Strands Agents with an MCP server that has many tools
6. Adding tool search results to a Strands Agent
7. Showing 3x latency improvement by using tool search

# Understanding fundamentals of AgentCore Gateway Search

When you create an AgentCore Gateway, you have the option to indicate that you want Search enabled.
For Gateways with search enabled, three things happen:

1. **Vector store is created**. The Gateway service automatically creates a serverless fully-managed vector store for your new Gateway. This enables a full semantic search across your Gateway tools. 
3. **Vector store is populated**. As you add Gateway Targets to your Gateway, the service automatically uses embeddings behind the scenes to populate the vector store based on the tools from the new Target. The tool metadata comes from the JSON defintions of your tools or the OpenAPI Schema specification for your REST services targets.
2. **Search tool (MCP based) is provided**. In addition to all of your user defined tools (from AWS Lambda targets or REST services), the Gateway gets one additional MCP tool that provides semantic search. It is named `x-amz-bedrock-agentcore-search`. The prefix ensures there are no name clashes with your user-defined tools. We may add more tools like that in the future as well. The search tool has a single argument called `query`. When the search tool is invoked, the Gateway service performs a semantic search using that query, matching it against available tool metadata (names, descriptions, input and output schema), and returns the most relevant tools in descending order of relevance.

# Preparing the notebook environment

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

Import all the required Python libraries, and load environment variables

In [31]:
from strands import Agent
from strands.models import BedrockModel
from strands.handlers import null_callback_handler

from strands.tools.mcp.mcp_client import MCPClient, MCPAgentTool

from mcp.client.streamable_http import streamablehttp_client
from mcp.types import Tool as MCPTool

import logging
import time
import json
import boto3
import requests
import utils

GATEWAY_NAME = "gateway-search-tutorial"

Set up a logger

In [32]:
# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.ERROR)  # INFO) #DEBUG) #

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", handlers=[logging.StreamHandler()]
)

Check our boto3 version

In [33]:
boto3.__version__

'1.41.2'

Get our boto3 client for the AgentCore control plane API.

In [37]:
session = boto3.Session()
agentcore_client = session.client(
    "bedrock-agentcore-control",
    region_name=aws_region
)

# Setting up a Gateway that has hundreds of tools

AgentCore Gateway provides a secure and scalable way to expose a curated set of existing APIs
as MCP tools for your agents. In a production setting, your Gateway resources would be created
using infrastructure as code with tools like CloudFormation, CDK, or Terraform. In this tutorial,
we use the boto3 control plane APIs directly, so that you can understand the resources and APIs more effectively.
This will help you more easily get started building and using your own gateways, and making more powerful
and secure agents.

At a high level, the steps for setting up your Gateway are:

1. Define what identity providers and credential providers you are using for inbound (agents calling Gateways) and outbound (Gateways calling tools) security.
2. Create the Gateway using `create_gateway`.
3. Add Gateway Targets using `create_gateway_target`, to expose MCP tools that will be implemented in AWS Lambda or in existing RESTful services.

In this tutorial, we will use Amazon Cognito as the identity provider (IdP), AWS Lambda functions as targets, and AWS IAM for outbound authentication. The same concepts demonstrated in this tutorial still apply when using other IdP's or other target types.

### Creating Amazon Cognito resources

In this tutorial, we assume you have already created the following resources and have set up corresponding environment variables:

- IAM role for AWS Lambda execution (`gateway_lambda_iam_role`)
- AWS Lambda function for your simple math tools (`calc_lambda_arn`)
- AWS Lambda function for your restaurant reservation tool (`restaurant_lambda_arn`)
- Amazon Cognito user pools, giving you a client id (`cognito_client_id`) and a discovery URL (`cognito_discovery_url`)

Lets take a look at the JSON tool metadata for the restaurant API. Note that if we were integrating with existing REST services, the API specs would be provided using OpenAPI Schema instead.

In [38]:
with open("./restaurant/restaurant-api.json") as f:
    data = json.load(f)
print(json.dumps(data, indent=4))

[
    {
        "name": "create_booking",
        "description": "Create a new booking at restaurant_name",
        "inputSchema": {
            "type": "object",
            "properties": {
                "date": {
                    "type": "string",
                    "description": "The date of the booking in the format YYYY-MM-DD. Do NOT accept relative dates like today or tomorrow. Ask for today's date for relative date."
                },
                "hour": {
                    "type": "string",
                    "description": "the hour of the booking in the format HH:MM"
                },
                "restaurant_name": {
                    "type": "string",
                    "description": "name of the restaurant handling the reservation"
                },
                "guest_name": {
                    "type": "string",
                    "description": "The name of the customer to have in the reservation"
                },
                "num_gues

Here are the simple calculator APIs.

In [39]:
with open("./calc/calc-api.json") as f:
    data = json.load(f)[0:3]
print(json.dumps(data, indent=4))

[
    {
        "name": "add_numbers",
        "description": "Adds firstNumber and secondNumber together to get the sum (firstNumber + secondNumber)",
        "inputSchema": {
            "type": "object",
            "properties": {
                "firstNumber": {
                    "type": "number",
                    "description": "first number to add"
                },
                "secondNumber": {
                    "type": "number",
                    "description": "second number to add"
                }
            },
            "required": [
                "firstNumber",
                "secondNumber"
            ]
        }
    },
    {
        "name": "subtract_numbers",
        "description": "Subtracts the subtrahend from the minuend to find the difference (minuend - subtrahend)",
        "inputSchema": {
            "type": "object",
            "properties": {
                "minuend": {
                    "type": "number",
                    "descripti

Here is the AWS Lambda function implementation for the calculator tools.

In [40]:
from IPython.display import display, Code

with open("./calc/lambda_function_code.py", "r") as f:
    code_content = f.read()
display(Code(code_content, language="python"))

def get_named_parameter(event, name):
    return event[name]


def handle_add(event):
    firstNumber = int(get_named_parameter(event, "firstNumber"))
    secondNumber = int(get_named_parameter(event, "secondNumber"))
    return {"sum": firstNumber + secondNumber}


def handle_multiply(event):
    multiplicand = int(get_named_parameter(event, "multiplicand"))
    multiplier = int(get_named_parameter(event, "multiplier"))
    return {"product": multiplicand * multiplier}


def handle_divide(event):
    divisor = int(get_named_parameter(event, "divisor"))
    dividend = int(get_named_parameter(event, "dividend"))

    if divisor == 0:
        raise Exception("Divisor cannot be 0")

    quotient = dividend / divisor

    return {"quotient": quotient}


def handle_subtract(event):
    minuend = int(get_named_parameter(event, "minuend"))
    subtrahend = int(get_named_parameter(event, "subtrahend"))

    difference = minuend - subtrahend

    return {"difference": difference}


def lambda_handler(event, context):
    print(f"event: {event}")
    print(f"context: {context}")
    print(f"context.client_context: {context.client_context}")

    extended_tool_name = context.client_context.custom["bedrockAgentCoreToolName"]
    tool_name = extended_tool_name.split("___")[1]

    print(f"tool_name: {tool_name}")

    if tool_name == "add_numbers":
        result = handle_add(event)
    elif tool_name == "multiply_numbers":
        result = handle_multiply(event)
    elif tool_name == "divide_numbers":
        result = handle_divide(event)
    elif tool_name == "subtract_numbers":
        result = handle_subtract(event)
    else:
        result = f"Unrecognized tool_name: {tool_name}"

    print(f"result: {result}")
    return result

In [41]:
with open("./restaurant/lambda_function_code.py", "r") as f:
    code_content = f.read()
display(Code(code_content, language="python"))

def get_named_parameter(event, name):
    return event[name]


def handle_create_booking(event):
    bookingDate = get_named_parameter(event, "date")
    bookingHour = get_named_parameter(event, "hour")
    restaurantName = get_named_parameter(event, "restaurant_name")
    guestName = get_named_parameter(event, "guest_name")
    numGuests = int(get_named_parameter(event, "num_guests"))
    return f"Booking id 12345, for {numGuests} guests at {restaurantName} on {bookingDate} at {bookingHour} for {guestName} created."


def lambda_handler(event, context):
    print(f"event: {event}")
    print(f"context: {context}")
    print(f"context.client_context: {context.client_context}")

    extended_tool_name = context.client_context.custom["bedrockAgentCoreToolName"]
    tool_name = extended_tool_name.split("___")[1]

    print(f"tool_name: {tool_name}")

    if tool_name == "create_booking":
        result = handle_create_booking(event)
    else:
        result = f"Unrecognized tool_name: {tool_name}"

    print(f"result: {result}")
    return result

In [50]:
#### Create a sample AWS Lambda function that you want to convert into MCP tools
calc_lambda_resp = utils.create_gateway_lambda(
    "calc/lambda_function_code.zip", lambda_function_name="calc_lambda_gateway"
)

if calc_lambda_resp is not None:
    if calc_lambda_resp["exit_code"] == 0:
        print(
            "Lambda function created with ARN: ",
            calc_lambda_resp["lambda_function_arn"],
        )
    else:
        print(
            "Lambda function creation failed with message: ",
            calc_lambda_resp["lambda_function_arn"],
        )

Reading code from zip file
Creating IAM role for lambda function
Attaching policy to the IAM role
Role 'calc_lambda_gateway_lambda_iamrole' created successfully: arn:aws:iam::684359859465:role/calc_lambda_gateway_lambda_iamrole
Creating lambda function
Lambda function created with ARN:  arn:aws:lambda:us-west-2:684359859465:function:calc_lambda_gateway


In [51]:
calc_lambda_resp["lambda_function_arn"]

'arn:aws:lambda:us-west-2:684359859465:function:calc_lambda_gateway'

In [52]:
#### Create a sample AWS Lambda function that you want to convert into MCP tools
restaurant_lambda_resp = utils.create_gateway_lambda(
    "restaurant/lambda_function_code.zip",
    lambda_function_name="restaurant_lambda_gateway",
)

if restaurant_lambda_resp is not None:
    if restaurant_lambda_resp["exit_code"] == 0:
        print(
            "Lambda function created with ARN: ",
            restaurant_lambda_resp["lambda_function_arn"],
        )
    else:
        print(
            "Lambda function creation failed with message: ",
            restaurant_lambda_resp["lambda_function_arn"],
        )

Reading code from zip file
Creating IAM role for lambda function
Attaching policy to the IAM role
Role 'restaurant_lambda_gateway_lambda_iamrole' created successfully: arn:aws:iam::684359859465:role/restaurant_lambda_gateway_lambda_iamrole
Creating lambda function
Lambda function created with ARN:  arn:aws:lambda:us-west-2:684359859465:function:restaurant_lambda_gateway


In [53]:
restaurant_lambda_resp["lambda_function_arn"]

'arn:aws:lambda:us-west-2:684359859465:function:restaurant_lambda_gateway'

In [54]:
cognito_response = utils.setup_cognito_user_pool()

Creating Cognito User Pool: MCPServerPool
User Pool created with ID: us-west-2_eQysMf8zQ
Creating Cognito App Client: MCPServerPoolClient
App Client created with ID: 60m9f2tj57r5l441i5pkmr2lf0
Creating Cognito user: testuser
Setting permanent password for user: testuser
Pool ID: us-west-2_eQysMf8zQ
Discovery URL: https://cognito-idp.us-west-2.amazonaws.com/us-west-2_eQysMf8zQ/.well-known/openid-configuration
Client ID: 60m9f2tj57r5l441i5pkmr2lf0


In [55]:
bearer_token = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)

Authenticating user: testuser
Bearer token obtained successfully


In [56]:
gateway_role_arn = utils.create_gateway_iam_role(
    lambda_arns=[
        calc_lambda_resp["lambda_function_arn"],
        restaurant_lambda_resp["lambda_function_arn"],
    ]
)

Creating IAM role: GatewaySearchAgentCoreRole
Attaching policy: BedrockAgentPolicy
Gateway IAM role created successfully: arn:aws:iam::684359859465:role/GatewaySearchAgentCoreRole


#### Let's create a few helper functions for using the control plane APIs

In [57]:
def read_apispec(json_file_path):
    try:
        # read json file and return contents as string
        with open(json_file_path, "r") as file:
            # Parse JSON to Python object
            api_spec = json.load(file)
            return api_spec

    except FileNotFoundError:
        return f"Error: File {json_file_path} not found"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"


def list_gateways():
    response = agentcore_client.list_gateways()
    print(json.dumps(response, indent=2, default=str))
    return response

#### Create Gateway helper function
Here is a helper function for creating an AgentCore Gateway given a name and
description. It uses Amazon Cognito as its IdP, and pulls the allowed client ID
and the discovery URL from environment variables, as those were already defined.
It also defaults to enabling semantic search on the resulting Gateway, and uses
a predefined IAM role.

In [58]:
def create_gateway(gateway_name, gateway_desc):
    # Use Cognito for Inbound OAuth to our Gateway
    auth_config = {
        "customJWTAuthorizer": {
            "allowedClients": [cognito_response["client_id"]],
            "discoveryUrl": cognito_response["discovery_url"],
        }
    }
    # Enable semantic search of tools
    search_config = {
        "mcp": {"searchType": "SEMANTIC", "supportedVersions": ["2025-03-26"]}
    }
    # Create the gateway
    response = agentcore_client.create_gateway(
        name=gateway_name,
        roleArn=gateway_role_arn,
        authorizerType="CUSTOM_JWT",
        description=gateway_desc,
        protocolType="MCP",
        authorizerConfiguration=auth_config,
        protocolConfiguration=search_config,
    )
    # print(json.dumps(response, indent=2, default=str))
    return response["gatewayId"]

#### Create Gateway Target helper function
This function creates a new AWS Lambda target on an existing Gateway.
Simply provide the gateway ID, the name and description of the new target,
the ARN of the existing AWS Lambda function, and the JSON schema describing
the interfaces to the tools you want to expose from the gateway.

In [59]:
def create_gatewaytarget(gateway_id, target_name, target_descr, lambda_arn, api_spec):
    # Add a Lambda target to the gateway
    response = agentcore_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name=target_name,
        description=target_descr,
        targetConfiguration={
            "mcp": {
                "lambda": {
                    "lambdaArn": lambda_arn,
                    "toolSchema": {"inlinePayload": api_spec},
                }
            }
        },
        # Use IAM as credential provider
        credentialProviderConfigurations=[
            {"credentialProviderType": "GATEWAY_IAM_ROLE"}
        ],
    )
    return response["targetId"]

### Creating your first AgentCore Gateway
Before we setup your first Gateway, let's take a quick look at how 
Gateway provides security, both for inbound requests to use MCP tools,
and outbound access from the Gateway to tools and resources.

![How does it work](images/gateway_secure_access.png)

Now let's create the gateway for this tutorial, providing a name and a description.

In [60]:
print(f"Create gateway with name: {GATEWAY_NAME}")
gatewayId = create_gateway(
    gateway_name=GATEWAY_NAME, gateway_desc="AgentCore Gateway Tutorial"
)
print(f"Gateway created with id: {gatewayId}.")

Create gateway with name: gateway-search-tutorial
Gateway created with id: gateway-search-tutorial-4pslr9mhdk.


### Adding AgentCore Gateway Targets
In this tutorial, we assume you already have installed a pair of Lambda functions, one for doing simple
math calculations, and another that simulates creating a restaurant reservation. We'll add a Gateway Target
for each of these functions.

Once we've added those targets, we'll add additional targets to simply drive higher MCP tool counts that
help us demonstrate the power of AgentCore Gateway search.

Now that the gateway is created, let's add a target for making restaurant reservations
via a Lambda function.

In [61]:
restaurant_api_spec = read_apispec("./restaurant/restaurant-api.json")
restaurant_lambda_arn = restaurant_lambda_resp["lambda_function_arn"]
print(f"Restaurant Lambda ARN: {restaurant_lambda_arn}")

restaurantTargetId = create_gatewaytarget(
    gateway_id=gatewayId,
    lambda_arn=restaurant_lambda_arn,
    target_name="FoodTools",
    target_descr="Restaurant Tools",
    api_spec=restaurant_api_spec,
)
print(f"RestaurantTarget created with id: {restaurantTargetId} on gateway: {gatewayId}")

Restaurant Lambda ARN: arn:aws:lambda:us-west-2:684359859465:function:restaurant_lambda_gateway
RestaurantTarget created with id: Z3TAXCA1QG on gateway: gateway-search-tutorial-4pslr9mhdk


Here we'll add a second target, this time with a Lambda that implements 4 basic tools (add, subtract,
multiply, divide), and a set of 75 generated tool definitions for investment management (trading, credit research, quantitative analysis, portfolio management). The investment management tool definitions are not 
actually implemented in the Lambda function. We are only adding them to demonstrate a large volume of tools.

In [62]:
calc_api_spec = read_apispec("./calc/calc-api.json")
print(f"API spec for calc has {len(calc_api_spec)} functions\n")
calc_lambda_arn = calc_lambda_resp["lambda_function_arn"]
print(f"Calc Lambda ARN: {calc_lambda_arn}")

time.sleep(5)
calcTargetId = create_gatewaytarget(
    gateway_id=gatewayId,
    lambda_arn=calc_lambda_arn,
    target_name="CalcTools",
    target_descr="Calculation Tools",
    api_spec=calc_api_spec,
)
print(f"CalcTools Target created with id: {calcTargetId} on gateway: {gatewayId}")

API spec for calc has 79 functions

Calc Lambda ARN: arn:aws:lambda:us-west-2:684359859465:function:calc_lambda_gateway
CalcTools Target created with id: YPAIGU1EOD on gateway: gateway-search-tutorial-4pslr9mhdk


To demonstrate the power of gateway search, now we add a few more copies of the Calculator target, 
so that we end up with 300+ MCP tools exposed.

In [63]:
def add_more_tools(gatewayId):
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc2",
        target_descr="Calculation 2 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc2 Target created with id: {calcTargetId} on gateway: {gatewayId}")
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc3",
        target_descr="Calculation 3 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc3 Target created with id: {calcTargetId} on gateway: {gatewayId}")
    time.sleep(10)
    calcTargetId = create_gatewaytarget(
        gateway_id=gatewayId,
        lambda_arn=calc_lambda_arn,
        target_name="Calc4",
        target_descr="Calculation 4 Tools",
        api_spec=calc_api_spec,
    )
    print(f"Calc4 Target created with id: {calcTargetId} on gateway: {gatewayId}")

In [64]:
add_more_tools(gatewayId=gatewayId)

Calc2 Target created with id: BGQYKLSOCI on gateway: gateway-search-tutorial-4pslr9mhdk
Calc3 Target created with id: T8SE8DT2XX on gateway: gateway-search-tutorial-4pslr9mhdk
Calc4 Target created with id: MGYFRCW1T0 on gateway: gateway-search-tutorial-4pslr9mhdk


In [65]:
resp = agentcore_client.list_gateway_targets(gatewayIdentifier=gatewayId)
targets = resp["items"]
for target in resp["items"]:
    print(f"{target['name']} - {target['description']}")

Calc2 - Calculation 2 Tools
Calc4 - Calculation 4 Tools
Calc3 - Calculation 3 Tools
CalcTools - Calculation Tools
FoodTools - Restaurant Tools


# Searching for tools from a Gateway

### Getting familiar with MCP list tools before we search

Let's define some utility functions to retrieve our MCP endpoint URL for a given Gateway ID, and to 
retrieve our JWT OAuth access token to securely use our Gateway.

In [66]:
def get_gateway_endpoint(gateway_id):
    response = agentcore_client.get_gateway(gatewayIdentifier=gateway_id)
    gateway_url = response["gatewayUrl"]
    return gateway_url

Now that our Gateway is created and has targets, let's grab the MCP URL to that
Gateway. We can retrieve the endpoint URL from the Gateway control plane based on the Gateway ID.

#### Using MCP Inspector against your Gateway

Now that we have an endpoint URL for the MCP server, and we have a JWT bearer token, you may want to explore
the MCP server with the MCP Inspector tool. MCP Inspector is an open source tool that can connect to any MCP
server, lets you list the tools provided, and even provides an easy to use tool invocation experience. 

From your terminal window, simply enter `npx @modelcontextprotocol/inspector` to launch the MCP Inspector. Then paste
in your Gateway endpoint URL and your JWT token to connect. Once connected, try out List Tools and Invoke Tool.

Here's a sample screenshot.

![MCP Inspector](images/mcp_inspector.png)

In [67]:
gatewayEndpoint = get_gateway_endpoint(gateway_id=gatewayId)
print(f"Gateway Endpoint - MCP URL: {gatewayEndpoint}")

Gateway Endpoint - MCP URL: https://gateway-search-tutorial-4pslr9mhdk.gateway.bedrock-agentcore.us-west-2.amazonaws.com/mcp


MCP server security is based on OAuth. To interact with our Gateway, we'll need to
retrieve a JWT OAuth access token from our IdP.

In [68]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
print(f"Bearer token: {jwtToken}")

Authenticating user: testuser
Bearer token obtained successfully
Bearer token: eyJraWQiOiJYVnp2TXE5ZElQWHNiZ081SnNieFYwc1JwZDFlSms4RmYremVsaDJ1QndFPSIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiI2OGIxODNlMC1lMDAxLTcwZGMtNjE5OS0yNmQxZDYxMGQxNTYiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtd2VzdC0yLmFtYXpvbmF3cy5jb21cL3VzLXdlc3QtMl9lUXlzTWY4elEiLCJjbGllbnRfaWQiOiI2MG05ZjJ0ajU3cjVsNDQxaTVwa21yMmxmMCIsIm9yaWdpbl9qdGkiOiIzMWZiY2Q3YS05M2I1LTQxODYtYjdjOC1jZmQyODc2ZWNhYzciLCJldmVudF9pZCI6ImIyOWM1NzM3LTU1NjMtNDc1Zi1iZGZmLTI1YzVhYmRkM2NmYiIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbmluLnVzZXIuYWRtaW4iLCJhdXRoX3RpbWUiOjE3NjM4MzM2NzAsImV4cCI6MTc2MzgzNzI3MCwiaWF0IjoxNzYzODMzNjcwLCJqdGkiOiI5MzRhYzVjOC1kMGFjLTQxNWUtYTZiYS00ZDEzOWRkYWQ3OTMiLCJ1c2VybmFtZSI6InRlc3R1c2VyIn0.VP4yJ5K5Uq-HkPFzBjvJ1mtSlQjCjSso-zldDQE7buwPKc9Xm2vBP9DV7fdTD7XD36gbZeOiUGrIsn4g6YALMCnEp40vtcP0iVAJK25DZ5s0HK66gEcDqLWXILS5DFnVODPyEy1qr3DwxlagsSWq14c1dIalmYk0fMSRTBd9RbbkabSQ7woJ5iFfygSmM8s05gZFSGDaRSPN5qzfumFKWMtQCxriVy-Yf_V9O

In [ ]:
# !npx @modelcontextprotocol/inspector

#### Creating helper functions that use jsonrpc to invoke MCP tools or list them
Let's define a helper function called `invoke_gateway_tool` that uses jsonrpc to invoke any of the
MCP tools exposed by an MCP Server, including of course, your Gateway. Given an endpoint URL and a JWT token,
you can use this utility to invoke any of the MCP tools that AgentCore Gateway made available
for you when you added Gateway Targets to your Gateway.

In [69]:
def invoke_gateway_tool(gateway_endpoint, jwt_token, tool_params):
    # print(f"Invoking tool {tool_params['name']}")

    requestBody = {
        "jsonrpc": "2.0",
        "id": 2,
        "method": "tools/call",
        "params": tool_params,
    }
    response = requests.post(
        gateway_endpoint,
        json=requestBody,
        headers={
            "Authorization": f"Bearer {jwt_token}",
            "Content-Type": "application/json",
        },
    )

    return response.json()

Here's another utility function for using MCP's `tools/list` method for listing the MCP tools
available from your Gateway. Given a Gateway ID and a JWT Token, it retrieves the full set
of tools from that Gateway, and returns a list in agent-ready form. The returned list contains 
Strands Agents MCPAgentTool objects that are suitable for handing your Agent. 

Note that `tools/list` call is paginated, so the function needs to loop, getting a page of
tools at a time, until the `nextCursor` field is no longer populated. The utility function directly
calls the endpoint using HTTPS and the jsonrpc protocol. This is a lower level way to list tools
compared to the `MCPClient` class provided by Strands Agents. We'll see that experience later.

In [70]:
def get_all_agent_tools_from_mcp_endpoint(gateway_endpoint, jwt_token, client):
    more_tools = True
    tools_count = 0
    tools_list = []

    requestBody = {"jsonrpc": "2.0", "id": 2, "method": "tools/list", "params": {}}
    next_cursor = ""

    while more_tools:
        if tools_count == 0:
            requestBody["params"] = {}
        else:
            print(f"\nGetting next page of tools since a next cursor was returned\n")
            requestBody["params"] = {"cursor": next_cursor}

        headers = {
            "Authorization": f"Bearer {jwt_token}",
            "Content-Type": "application/json",
        }

        print(f"\n\nListing tools for gateway {gateway_endpoint}")

        response = requests.post(gateway_endpoint, json=requestBody, headers=headers)

        tools_json = response.json()
        tools_count += len(tools_json["result"]["tools"])

        for tool in tools_json["result"]["tools"]:
            mcp_tool = MCPTool(
                name=tool["name"],
                description=tool["description"],
                inputSchema=tool["inputSchema"],
            )
            mcp_agent_tool = MCPAgentTool(mcp_tool, client)
            short_descr = tool["description"][0:40] + "..."
            print(f"adding tool '{mcp_agent_tool.tool_name}' - {short_descr}")
            tools_list.append(mcp_agent_tool)

        if "nextCursor" in tools_json["result"]:
            next_cursor = tools_json["result"]["nextCursor"]
            more_tools = True
        else:
            more_tools = False

    print(f"\nTotal tools found: {tools_count}\n")
    return tools_list

Lets use this helper function and see the results.

In [71]:
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_agent_tools_from_mcp_endpoint(
        gateway_endpoint=gatewayEndpoint, jwt_token=jwtToken, client=client
    )
    print(f"\nFound {len(all_tools)} tools using jsonrpc to list MCP tools\n")



Listing tools for gateway https://gateway-search-tutorial-4pslr9mhdk.gateway.bedrock-agentcore.us-west-2.amazonaws.com/mcp
adding tool 'x_amz_bedrock_agentcore_search' - A special tool that returns a trimmed do...
adding tool 'Calc2___add_numbers' - Adds firstNumber and secondNumber togeth...
adding tool 'Calc2___analyze_bond_liquidity' - Analyzes liquidity metrics for fixed inc...
adding tool 'Calc2___analyze_covenant_compliance' - Analyzes compliance with bond covenants ...
adding tool 'Calc2___analyze_debt_structure' - Analyzes the debt structure of an issuer...
adding tool 'Calc2___analyze_fund_flows' - Analyzes fund inflows and outflows over ...
adding tool 'Calc2___analyze_peer_comparison' - Compares a portfolio or fund against pee...
adding tool 'Calc2___analyze_portfolio_liquidity' - Analyzes the liquidity profile of a port...
adding tool 'Calc2___analyze_technical_indicators' - Calculates technical indicators for a se...
adding tool 'Calc2___analyze_time_series' - Performs t

#### Using Strands Agents list_tools_sync() with pagination
If you have written any Python based MCP client, you are likely familiar with the `list_tools_sync()` method 
which returns the set of tools available from the MCP Server which the client is associated
with. But did you know MCP list tools is also paginated? By default, you will only get the first small
subset of tools returned. For simple MCP servers, you may not have noticed this, but for many real world 
MCP servers, your code needs to loop, grabbing pages of tools at a time
until there are no more pages remaining. The following utility `get_all_mcp_tools_from_mcp_client` does exactly that. 
It returns the full list of tools from a given Strands Agent MCP Client.

In [72]:
def get_all_mcp_tools_from_mcp_client(client):
    more_tools = True
    tools = []
    pagination_token = None
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)
        tools.extend(tmp_tools)
        if tmp_tools.pagination_token is None:
            more_tools = False
        else:
            more_tools = True
            pagination_token = tmp_tools.pagination_token
    return tools

Let's give it a try with our Gateway and find out how many tools the Python client finds. 
First we create an MCPClient object based on our endpoint URL and our JWT bearer token. Then 
we retrieve the full set of tools across many pages of tools returned by the MCP server.
Given the targets we added earlier, this should return 300+ tools.

In [73]:
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")


Found 318 tools from list_tools_sync() on mcp client



We have now seen 3 different ways to get the full set of tools from your Gateway using it
as an MCP Server: 

1. directly using jsonrpc
2. using the `list_tools_sync()` method on the Strands Agent MCPClient
3. using the MCP Inspector tool (which uses jsonrpc behind the scenes). 

For typical developers building agents, you'll be using option 2.

### Using the built-in Gateway semantic search tool
Now lets try our first semantic search on the Gateway using its built-in search tool provided as
an additional MCP tool that gets added to your MCP tool list.

First let's define a simple utility function to execute the search tool using MCP.
Just like for listing tools, we need the gateway endpoint and JWT token. Other than that,
all we need to pass in is the search query. The Gateway search tool will do the rest,
matching that query against the serverless vector store that it automatically manages on your behalf.

In [74]:
def tool_search(gateway_endpoint, jwt_token, query):
    toolParams = {
        "name": "x_amz_bedrock_agentcore_search",
        "arguments": {"query": query},
    }
    toolResp = invoke_gateway_tool(
        gateway_endpoint=gateway_endpoint, jwt_token=jwt_token, tool_params=toolParams
    )
    tools = toolResp["result"]["structuredContent"]["tools"]
    return tools

In [75]:
start_time = time.time()
tools_found = tool_search(
    gateway_endpoint=gatewayEndpoint,
    jwt_token=jwtToken,
    query="find me 3 credit research tools",
)
end_time = time.time()
print(
    f"tool search via direct Gateway invocation took {(end_time - start_time):.2f} seconds"
)
print(f"Top tool: {tools_found[0]['name']}")

tool search via direct Gateway invocation took 1.99 seconds
Top tool: CalcTools___get_credit_rating_history


Notice how fast the search returns, in under a second in most cases. The results are returned
in descending order of search relevance based on matching the query to the tool metadata.
The most relevant tools are first on the list. The intial implementation of search gives back
up to 10 results. You could then use all of these tools in your agent, or simply pick a subset of
the most relevant matches.

# Using Strands Agents with an MCP server that has many tools

First, we select a model to use with our Strands Agent. 
For this notebook, we are using Amazon Bedrock models, but Strands and AgentCore
can work with any LLM.

In [76]:
bedrockmodel = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    temperature=0.7,
    streaming=True,
    boto_session=session,
)

#### Simple Strands Agent using AgentCore Gateway for agent tools
Now lets show how easy it is to use a Strands Agent to leverage an MCP Server
provided by AgentCore Gateway. In our
simple example, we ask the agent to add some numbers.

In [85]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")

    simple_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )
    result = simple_agent("add 100 plus 50 pass ")
    print(f"{result.message['content'][0]['text']}")
    print(f"{result.metrics.accumulated_usage}")

Authenticating user: testuser
Bearer token obtained successfully

Found 318 tools from list_tools_sync() on mcp client

The sum of 100 plus 50 is 150.
{'inputTokens': 100405, 'outputTokens': 114, 'totalTokens': 100519}


The Strands Agents framework also lets you bypass the agent event loop, invoking an MCP tool directly.
Since Gateway tools are exposed as native MCP tools, this can be done against Gateway tools as well. Here
we call a Gateway MCP tool using the `agent.tool.<tool_name>(args)` syntax:

```python
direct_result = simple_agent.tool.Calc2___add_numbers(firstNumber=10, secondNumber=20)
resp_json = json.loads(direct_result['content'][0]['text'])
```

In [86]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")

    simple_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )
    direct_result = simple_agent.tool.Calc2___add_numbers(
        firstNumber=10, secondNumber=20
    )
    print(f"direct result = {direct_result}")

Authenticating user: testuser
Bearer token obtained successfully

Found 318 tools from list_tools_sync() on mcp client

direct result = {'status': 'success', 'toolUseId': 'tooluse_Calc2___add_numbers_911321584', 'content': [{'text': '{"sum":30}'}]}


In [87]:
def get_search_tool(client):
    mcp_tool = MCPTool(
        name="x_amz_bedrock_agentcore_search",
        description="A special tool that returns a trimmed down list of tools given a context. Use this tool only when there are many tools available and you want to get a subset that matches the provided context.",
        inputSchema={
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "search query to use for finding tools",
                }
            },
            "required": ["query"],
        },
    )
    return MCPAgentTool(mcp_tool, client)

In [88]:
def search_using_strands(client, query):
    simple_agent = Agent(
        model=bedrockmodel,
        tools=[get_search_tool(client)],
        callback_handler=null_callback_handler,
    )

    direct_result = simple_agent.tool.x_amz_bedrock_agentcore_search(query=query)

    resp_json = json.loads(direct_result["content"][0]["text"])
    search_results = resp_json["tools"]
    # print(json.dumps(search_results, indent=4))
    return search_results

In [89]:
def find_strands_tools(client, query, top_n):
    strands_mcp_tools = []
    results = search_using_strands(client, query)
    for tool in results[:top_n]:
        mcp_tool = MCPTool(
            name=tool["name"],
            description=tool["description"],
            inputSchema=tool["inputSchema"],
        )
        strands_mcp_tools.append(MCPAgentTool(mcp_tool, client))
    return strands_mcp_tools

In [90]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    simple_agent = Agent(
        model=bedrockmodel,
        tools=[get_search_tool(client)],
        callback_handler=null_callback_handler,
    )

    direct_result = simple_agent.tool.x_amz_bedrock_agentcore_search(
        query="find equity trading tools"
    )

    resp_json = json.loads(direct_result["content"][0]["text"])
    search_results = resp_json["tools"]
    print(json.dumps(search_results, indent=4))

Authenticating user: testuser
Bearer token obtained successfully
[
    {
        "inputSchema": {
            "type": "object",
            "properties": {
                "technicalCriteria": {
                    "type": "object",
                    "properties": {
                        "rsiMax": {
                            "type": "number"
                        },
                        "rsiMin": {
                            "type": "number"
                        },
                        "priceAboveSMA50": {
                            "type": "boolean"
                        },
                        "volumeIncreasePercent": {
                            "type": "number"
                        }
                    }
                },
                "universe": {
                    "type": "string"
                },
                "fundamentalCriteria": {
                    "type": "object",
                    "properties": {
                        "peRatioM

In [91]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    results = search_using_strands(client, "find trading tools")
    print(json.dumps(search_results[0], indent=4))

    results = search_using_strands(client, "find credit research tools")
    print(json.dumps(search_results[0], indent=4))

Authenticating user: testuser
Bearer token obtained successfully
{
    "inputSchema": {
        "type": "object",
        "properties": {
            "technicalCriteria": {
                "type": "object",
                "properties": {
                    "rsiMax": {
                        "type": "number"
                    },
                    "rsiMin": {
                        "type": "number"
                    },
                    "priceAboveSMA50": {
                        "type": "boolean"
                    },
                    "volumeIncreasePercent": {
                        "type": "number"
                    }
                }
            },
            "universe": {
                "type": "string"
            },
            "fundamentalCriteria": {
                "type": "object",
                "properties": {
                    "peRatioMax": {
                        "type": "number"
                    },
                    "debtToEquityMax": {
  

# Adding tool search results to a Strands Agent

Now let's look at how the tools returned from a search can be added to a
Strands Agent. To make the coding simpler, let's provide a utility function that
maps tool search results to Strands MCPAgentTool objects. Simply pass in the 
search results, and indicate how many of those results you want to pass to your
agent.

In [92]:
def tools_to_strands_mcp_tools(tools, top_n):
    strands_mcp_tools = []
    for tool in tools[:top_n]:
        mcp_tool = MCPTool(
            name=tool["name"],
            description=tool["description"],
            inputSchema=tool["inputSchema"],
        )
        strands_mcp_tools.append(MCPAgentTool(mcp_tool, client))
    return strands_mcp_tools

In [94]:
jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    agent = Agent(
        model=bedrockmodel,
        tools=find_strands_tools(
            client,
            "tools for doing addition, subtraction, multiplication, division",
            10,
        ),
    )
    result = agent("(10*2)/(5-3)")
    print(f"{result.message['content'][0]['text']}")
    print(f"{result.metrics.accumulated_usage}")

Authenticating user: testuser
Bearer token obtained successfully
I'll solve this expression step by step.

First, let's calculate the multiplication inside the parentheses (10*2):
Tool #1: CalcTools___multiply_numbers


WARNING | client | Tool CalcTools___multiply_numbers not listed by server, cannot validate any structured content


Next, let's calculate the subtraction in the denominator (5-3):
Tool #2: CalcTools___subtract_numbers


WARNING | client | Tool CalcTools___subtract_numbers not listed by server, cannot validate any structured content


Now, let's divide the results (20 ÷ 2):
Tool #3: CalcTools___divide_numbers


WARNING | client | Tool CalcTools___divide_numbers not listed by server, cannot validate any structured content


Therefore, (10*2)/(5-3) = 10Therefore, (10*2)/(5-3) = 10
{'inputTokens': 6401, 'outputTokens': 314, 'totalTokens': 6715}


In [95]:
%%time

jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)
with client:
    print("Searching for an ADDING tool from endpoint with full set of tools...")
    tools_found = tool_search(
        gateway_endpoint=gatewayEndpoint,
        jwt_token=jwtToken,
        query="tools for multiplying two numbers",
    )
    print(f"Top tool found: {tools_found[0]['name']}\n")

    agent = Agent(model=bedrockmodel, tools=tools_to_strands_mcp_tools(tools_found, 1))
    result = agent("10 * 70")
    print(f"{result.message['content'][0]['text']}")
    print(f"{result.metrics.accumulated_usage}")

Authenticating user: testuser
Bearer token obtained successfully
Searching for an ADDING tool from endpoint with full set of tools...
Top tool found: CalcTools___multiply_numbers

I'll calculate 10 * 70 for you using the multiplication tool.
Tool #1: CalcTools___multiply_numbers


WARNING | client | Tool CalcTools___multiply_numbers not listed by server, cannot validate any structured content


The result of multiplying 10 by 70 is 700.The result of multiplying 10 by 70 is 700.
{'inputTokens': 979, 'outputTokens': 114, 'totalTokens': 1093}
CPU times: user 167 ms, sys: 27.9 ms, total: 195 ms
Wall time: 10.1 s


Notice the latency improvement. This example using a subset of tools from Gateway search is significantly faster than
agent invocation when depending on hundreds of tools.

# Showing 3x latency improvement by using tool search

Now that we know how to use Gateway MCP tools from a Strands agent, and we know how to search for tools and
add them to an agent, lets show the power of search. We'll highlight the significant latency reduction
and input token usage that can be delivered.

To demonstrate the latency and token reductions, we compare 2 approaches side by side:

1. **Without search**. We add the full set of MCP tools that the MCP server exposes (300+ in our case) to our agent and let the agent do its tool selection and invocation accordingly.
2. **Using search**. In the second approach, we do a search based on the topic at hand, and only send in the most relevant tools to the agent. To prove the point, we use two different topics: math (adding numbers), and food (booking a restaurant reservation), each requiring a different set of tools.

To normalize the latency distribution and get a meaningful comparison, we perform multiple iterations of
each approach. Also, to avoid overstating the gains, when doing the search approach, we include not only the
latency of the agent invocation, but also the latency of performing the tool search. For each 
iteration, we hand the agent two tasks: 

1. Math task -  add 2 numbers 
2. Food task - book a restaurant reservation

The results below demonstrate the benefits, highlighting 3x latency reduction, and even greater reduction in
input token usage. Note that while token usage savings translate to cost savings, that may not be as impactful due to
the relatively lower cost of input tokens (for many model providers, input tokens are much lest costly). Even so, for 
large scale agent deployment, even input token usage costs can add up, so dynamic search can help reduce 
agent runtime costs as well.

#### Measure latency and token usage for agent using the entire set of MCP tools

In [96]:
iterations = 2
full_tokens = light_tokens = 0
full_elapsed_time = light_elapsed_time = 0

jwtToken = utils.get_bearer_token(
    client_id=cognito_response["client_id"],
    username="testuser",
    password="MyPassword123!",
)
client = MCPClient(
    lambda: streamablehttp_client(
        f"{gatewayEndpoint}", headers={"Authorization": f"Bearer {jwtToken}"}
    )
)

Authenticating user: testuser
Bearer token obtained successfully


In [97]:
with client:
    all_tools = get_all_mcp_tools_from_mcp_client(client)
    print(f"\nFound {len(all_tools)} tools from list_tools_sync() on mcp client\n")
    heavy_agent = Agent(
        model=bedrockmodel, tools=all_tools, callback_handler=null_callback_handler
    )

    math_input = "add 100 plus <iteration>"
    food_input = (
        "book me a table for 2 at Burger King under name Jo Smith at 7pm August <day>"
    )

    print("using agent with ALL tools...")
    start_time = time.time()

    for i in range(iterations):
        result = heavy_agent(math_input.replace("<iteration>", str(i + 1)))
        print(f"{i+1}) {result.message['content'][0]['text']}")

        result = heavy_agent(food_input.replace("<day>", str(i + 1)))
        print(f"{i+1}) {result.message['content'][0]['text']}")

    end_time = time.time()
    full_tokens = result.metrics.accumulated_usage["totalTokens"]
    full_elapsed_time = end_time - start_time
    print(f"\nTotal time: {full_elapsed_time:.1f} s, tokens: {full_tokens:,d}\n")


Found 318 tools from list_tools_sync() on mcp client

using agent with ALL tools...
1) The sum of 100 plus 1 is 101.
1) Your reservation has been successfully created! 

Here are the details:
- Restaurant: Burger King
- Date: August 1, 2024
- Time: 7:00 PM
- Number of guests: 2
- Name: Jo Smith
- Booking confirmation ID: 12345

Your table is all set for August 1st. Enjoy your meal!
2) The sum of 100 plus 2 is 102.
2) Your reservation has been successfully created! 

Here are the details:
- Restaurant: Burger King
- Date: August 2, 2024
- Time: 7:00 PM
- Number of guests: 2
- Name: Jo Smith
- Booking confirmation ID: 12345

Your table is all set for August 2nd. Enjoy your meal!

Total time: 53.5 s, tokens: 405,052



#### Measure latency and token usage for agents using Gateway Search
Now we'll use a dynamic approach, calling search to find relevant tools, and then calling the
agent with only those relevant tools. Note that since we are resetting the agent on each 
conversation turn, we're also intializing the message list from conversation history of the prior turn.

In [98]:
with client:
    print("using agent with ONLY tools from focused search...")
    start_time = time.time()
    messages = []

    light_agent = Agent()

    for i in range(iterations):
        print("Searching for an ADDING tool from endpoint with full set of tools...")
        tools_found = tool_search(
            gateway_endpoint=gatewayEndpoint,
            jwt_token=jwtToken,
            query="tools for simply adding two numbers",
        )
        print(f"Top tool found: {tools_found[0]['name']}\n")
        light_agent = Agent(
            model=bedrockmodel,
            tools=tools_to_strands_mcp_tools(tools_found, 1),
            messages=messages,
            callback_handler=null_callback_handler,
        )
        light_result = light_agent(math_input.replace("<iteration>", str(i + 1)))
        print(f"{i+1}) {light_result.message['content'][0]['text']}")
        messages = light_agent.messages

        print(
            "Searching for a RESTAURANT BOOKING tool from endpoint with full set of tools..."
        )
        tools_found = tool_search(
            gateway_endpoint=gatewayEndpoint,
            jwt_token=jwtToken,
            query="tools for booking a restaurant reservation",
        )
        print(f"Top tool found: {tools_found[0]['name']}\n")
        light_agent = Agent(
            model=bedrockmodel,
            tools=tools_to_strands_mcp_tools(tools_found, 1),
            messages=messages,
            callback_handler=null_callback_handler,
        )
        light_result = light_agent(food_input.replace("<day>", str(i + 1)))
        print(f"{i+1}) {light_result.message['content'][0]['text']}")
        messages = light_agent.messages
        light_tokens = light_result.metrics.accumulated_usage["totalTokens"]
    end_time = time.time()

    light_elapsed_time = end_time - start_time
    print(f"\nTotal time: {light_elapsed_time:.1f} s, tokens: {light_tokens:,d}\n")

using agent with ONLY tools from focused search...
Searching for an ADDING tool from endpoint with full set of tools...
Top tool found: CalcTools___add_numbers



WARNING | client | Tool CalcTools___add_numbers not listed by server, cannot validate any structured content


1) The sum of 100 plus 1 is 101.
Searching for a RESTAURANT BOOKING tool from endpoint with full set of tools...
Top tool found: FoodTools___create_booking



WARNING | client | Tool FoodTools___create_booking not listed by server, cannot validate any structured content


1) Great! Your reservation has been confirmed. Here are the details:

- Restaurant: Burger King
- Date: August 1, 2023
- Time: 7:00 PM
- Number of guests: 2
- Reservation name: Jo Smith
- Booking ID: 12345

Your table has been successfully booked. Is there anything else you need help with?
Searching for an ADDING tool from endpoint with full set of tools...
Top tool found: CalcTools___add_numbers



WARNING | client | Tool CalcTools___add_numbers not listed by server, cannot validate any structured content


2) The sum of 100 plus 2 is 102.
Searching for a RESTAURANT BOOKING tool from endpoint with full set of tools...
Top tool found: FoodTools___create_booking



WARNING | client | Tool FoodTools___create_booking not listed by server, cannot validate any structured content


2) Great! Your reservation has been confirmed. Here are the details:

- Restaurant: Burger King
- Date: August 2, 2023
- Time: 7:00 PM
- Number of guests: 2
- Reservation name: Jo Smith
- Booking ID: 12345

Your table has been successfully booked. Is there anything else you need help with?

Total time: 32.1 s, tokens: 2,743



#### Compare results, higlighting benefits of search

In [ ]:
print(
    f"\n\nLatency without search: {full_elapsed_time:.1f}s, using search: {light_elapsed_time:.1f}s"
)
print(f"Tokens without search: {full_tokens:,d}, using search: {light_tokens:,d}")

# Conclusion
In this tutorial, you have learned about Amazon Bedrock AgentCore Gateway and its built-in 
fully managed semantic search capability. You have seen the following:

- how to create a gateway with semantic search enabled
- how to add multiple gateway targets to surface 300+ MCP tools from a single endpoint
- how to list the tools on your gateway using 3 different approaches
- how to use the built-in semantic search tool to find relevant tools
- how to integrate search with your Strands Agent
- how to compare performance of an agent using a server with hundreds of tools versus one that uses semantic search to narrow tools to a specific topic

AgentCore Gateway search is helpful for more advanced use cases as well. By offering the search as a native
MCP tool and not just a control plane API, you can imagine giving your agents more autonomy to discover new
MCP servers, and find new capabilities at runtime leading to breakthroughs in solving more challenging problems.
In addition, search is an important foundation for MCP registries and supporting agent developers as they 
design and build new agents.

# Minimal AgentCore Memory Tutorial

This notebook demonstrates a **minimal**, clean example of using Amazon Bedrock **AgentCore Memory** with a **Strands Agent**, using only:

- A simple **summary-based memory strategy**
- A tiny **MemoryHookProvider** (retrieve + save)
- A simple **agent conversation** showing memory in action
- A **memory retrieval** step

You can merge or extend this with your other notebooks as needed.

## Step 1 — Install + Imports

In [ ]:
!pip install -q bedrock-agentcore-client strands

In [99]:
import os
from datetime import datetime
from strands import Agent
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType
from strands.hooks import HookProvider, HookRegistry, MessageAddedEvent, AfterInvocationEvent
import logging

logging.basicConfig(level=logging.INFO)

## Step 2 — Configuration

In [ ]:
import boto3
import json
import time

iam_client = boto3.client('iam')
role_name = "BedrockMemoryExecutionRole_Notebook"

trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {"Service": "bedrock-agentcore.amazonaws.com"},
            "Action": "sts:AssumeRole"
        }
    ]
}

permission_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": "*"
        }
    ]
}

try:
    response = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy)
    )
    role_arn = response['Role']['Arn']
    
    iam_client.put_role_policy(
        RoleName=role_name,
        PolicyName="MemoryModelInvokePolicy",
        PolicyDocument=json.dumps(permission_policy)
    )
    
    print(f"✅ Role created successfully!\nROLE_ARN = \"{role_arn}\"")

except iam_client.exceptions.EntityAlreadyExistsException:
    print(f"ℹ️ Role already exists. Updating trust policy...")
    
    iam_client.update_assume_role_policy(
        RoleName=role_name,
        PolicyDocument=json.dumps(trust_policy)
    )
    
    iam_client.put_role_policy(
        RoleName=role_name,
        PolicyName="MemoryModelInvokePolicy",
        PolicyDocument=json.dumps(permission_policy)
    )
    
    role_arn = iam_client.get_role(RoleName=role_name)['Role']['Arn']
    print(f"✅ Trust policy updated!\nROLE_ARN = \"{role_arn}\"")

except Exception as e:
    print(f"❌ Error creating/updating role: {e}")
    raise

print("Waiting for IAM role to propagate...")
time.sleep(10)

REGION = os.getenv('AWS_REGION', 'us-west-2')
ROLE_ARN = role_arn

ACTOR_ID = f"actor-{datetime.now().strftime('%Y%m%d%H%M%S')}"
SESSION_ID = f"session-{datetime.now().strftime('%Y%m%d%H%M%S')}"

ℹ️ Role already exists. Updating trust policy...
✅ Trust policy updated!
ROLE_ARN = "arn:aws:iam::684359859465:role/BedrockMemoryExecutionRole_Notebook"
Waiting for IAM role to propagate...


## Step 3 — Create Minimal Memory Resource
This memory stores short **summaries** of every user ↔ assistant turn.

In [114]:
CUSTOM_PROMPT = """
Your task is to extract math learning data from the user's conversations. You store the progress of the user in a memory system to understand their math level and help them progress.

You are tasked with analyzing conversations to extract the user's math learning patterns. You'll be analyzing two sets of data: 

<past_conversation> 
[Past conversations between the user and math tutor will be placed here for context] 
</past_conversation> 

<current_conversation> 
[The current conversation between the user and math tutor will be placed here] 
</current_conversation> 

Your job is to identify and categorize the user's math learning profile:
- Extract the user's current math level from problems they solve correctly/incorrectly
- Extract the user's preferred learning style from how they ask questions and respond to explanations
- Extract topic strengths and weaknesses from their performance patterns
- Track learning progress and identify areas needing reinforcement
"""

In [115]:
client = MemoryClient(region_name=aws_region)

strategies = [
    {
        StrategyType.CUSTOM.value: {
            "name": "CustomSemanticMemory",
            "description": "Captures facts from conversations",
            "namespaces": ["/students/math/{actorId}"],
            "configuration" : {
                "semanticOverride" : {
                    "extraction" : {
                        "modelId" : "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
                        "appendToPrompt": CUSTOM_PROMPT
                    }
                },
    }}}
]

memory = client.create_memory_and_wait(
    name="MinimalMemoryDemo",
    description="Demo memory",
    strategies=strategies,
    event_expiry_days=7,
    memory_execution_role_arn=ROLE_ARN
)

memory_id = memory['id']
memory_id

'MinimalMemoryDemo-sSODXMFQKA'

## Step 4 — Minimal MemoryHookProvider
Automatically:
- **Retrieve** summaries when user speaks
- **Store** summaries after assistant responds

This is intentionally concise.

In [ ]:
class SimpleMemoryHooks(HookProvider):
    def __init__(self, memory_id, client):
        self.memory_id = memory_id
        self.client = client

    def retrieve(self, event: MessageAddedEvent):
        messages = event.agent.messages
        if messages[-1]['role'] != 'user':
            return

        namespace = f"/demo/{event.agent.state.actor_id}"
        query = messages[-1]['content'][0].get('text','')

    
        try:
            results = self.client.retrieve_memories(
                memory_id=self.memory_id,
                namespace=namespace,
                query=query
            )
            if results:
                summary_texts = [m['content']['text'] for m in results]
                injected = "\n\nRelevant past summaries:\n- " + "\n- ".join(summary_texts)
                messages[-1]['content'][0]['text'] += injected
        except Exception as e:
            logging.error(f"Memory retrieval failed: {e}")

    def save(self, event: AfterInvocationEvent):
        msgs = event.agent.messages
        if len(msgs) < 2:
            return

        user_msg = msgs[-2]['content'][0]['text'] if msgs[-2]['role']=='user' else None
        assistant_msg = msgs[-1]['content'][0]['text'] if msgs[-1]['role']=='assistant' else None
        if not user_msg or not assistant_msg:
            return

        self.client.create_event(
            memory_id=self.memory_id,
            actor_id=event.agent.state.actor_id,
            session_id=event.agent.state.session_id,
            messages=[(user_msg, "USER"), (assistant_msg, "ASSISTANT")]
        )

    def register_hooks(self, registry: HookRegistry):
        registry.add_callback(MessageAddedEvent, self.retrieve)
        registry.add_callback(AfterInvocationEvent, self.save)


## Step 5 — Create Agent with Memory Hooks

In [120]:
hooks = SimpleMemoryHooks(memory_id, client)

agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt="You are a friendly personal assistant.",
    hooks=[hooks],
    state={"actor_id": ACTOR_ID, "session_id": SESSION_ID}
)

print("Agent ready.")

Agent ready.


## Step 6 — Demo Conversation

In [121]:
resp1 = agent("Hi, I'm learning about geometry. Can you help?")
resp1

AttributeError: 'AgentState' object has no attribute 'actor_id'

In [ ]:
resp2 = agent("What should I study next after triangles?")
resp2

## Step 7 — Retrieve Stored Memory

In [ ]:
results = client.retrieve_memories(
    memory_id=memory_id,
    namespace=f"/demo/{ACTOR_ID}",
    query="geometry"
)

results